In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib

from tqdm import tqdm
%matplotlib inline
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import transforms, utils

import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from tqdm import tqdm as tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic=True

cuda


In [2]:
class Focus(nn.Module):
  def __init__(self):
    super(Focus, self).__init__()

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0, bias=False)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=0, bias=False)
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=0, bias=False)
    self.fc1 = nn.Linear(1024, 512, bias=False)
    self.fc2 = nn.Linear(512, 64, bias=False)
    self.fc3 = nn.Linear(64, 10, bias=False)
    self.fc4 = nn.Linear(10,1, bias=False)

    torch.nn.init.xavier_normal_(self.conv1.weight)
    torch.nn.init.xavier_normal_(self.conv2.weight)
    torch.nn.init.xavier_normal_(self.conv3.weight)
    torch.nn.init.xavier_normal_(self.fc1.weight)
    torch.nn.init.xavier_normal_(self.fc2.weight)
    torch.nn.init.xavier_normal_(self.fc3.weight)
    torch.nn.init.xavier_normal_(self.fc4.weight)

  def forward(self,z):  #y is avg image #z batch of list of 9 images
    batch = z.size(0)
    patches = z.size(1)
    z = z.view(batch*patches,3,32,32)
    alpha =  self.helper(z)
    alpha = alpha.view(batch,patches,-1)
    return alpha[:,:,0] # scores 
    
  def helper(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    # print(x.shape)
    x = (F.relu(self.conv3(x)))
    x =  x.view(x.size(0), -1)
    # print(x.shape)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x
        

In [3]:
class Classification(nn.Module):
  def __init__(self):
    super(Classification, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
    self.fc4 = nn.Linear(10,3)

    torch.nn.init.xavier_normal_(self.conv1.weight)
    torch.nn.init.zeros_(self.conv1.bias)
    torch.nn.init.xavier_normal_(self.conv2.weight)
    torch.nn.init.zeros_(self.conv2.bias)
    torch.nn.init.xavier_normal_(self.fc1.weight)
    torch.nn.init.zeros_(self.fc1.bias)
    torch.nn.init.xavier_normal_(self.fc2.weight)
    torch.nn.init.zeros_(self.fc2.bias)
    torch.nn.init.xavier_normal_(self.fc3.weight)
    torch.nn.init.zeros_(self.fc3.bias)
    torch.nn.init.xavier_normal_(self.fc4.weight)
    torch.nn.init.zeros_(self.fc4.bias)

  def forward(self,z): 
    y1 = self.pool(F.relu(self.conv1(z)))
    y1 = self.pool(F.relu(self.conv2(y1)))
    y1 = y1.view(-1, 16 * 5 * 5)

    y1 = F.relu(self.fc1(y1))
    y1 = F.relu(self.fc2(y1))
    y1 = F.relu(self.fc3(y1))
    y1 = self.fc4(y1)
    return y1 

In [4]:
def calculate_loss_focus(gamma,focus_output):
    
    m = torch.nn.LogSoftmax(dim=1)
    log_outputs = m(focus_output)    
    loss_ = gamma*log_outputs
    loss_ = torch.sum(loss_,dim=1)
    loss_ = -torch.mean(loss_,dim=0)  
    return loss_ 
    

In [5]:
def calculate_loss_classification(gamma,classification_output,label,criterion,n_patches):
    #print(classification_output.shape)
    
    batch = label.size(0)
    classes = classification_output.size(2)
    label = label.repeat_interleave(n_patches)
    classification_output = classification_output.reshape((batch*n_patches,classes))
    loss_ = criterion(classification_output,label)
    
    loss_ = loss_.reshape((batch,n_patches))
    
    loss_ = gamma*loss_
    loss_ = torch.sum(loss_,dim=1)
    loss_ = torch.mean(loss_,dim=0)
    
    return loss_
    

In [6]:
def expectation_step(fc,cl,data,labels):
    batch= data.size(0)
    patches = data.size(1)
    with torch.no_grad():
        outputs_f = torch.softmax(fc(data),dim=1)
        data = data.reshape(batch*patches,3,32,32)
        outputs_g = cl(data)
        
    outputs_g = torch.softmax(outputs_g.reshape(batch,patches,3),dim=2)
    
        
        
    #print("Focus output",outputs_f.shape,torch.sum(outputs_f,dim=1))
    #print("Classification output",outputs_g.shape,torch.sum(outputs_g,dim=1),torch.sum(outputs_g,dim=2))
    outputs_g = outputs_g[np.arange(batch),:,labels]
    p_x_y_z = outputs_f*outputs_g   #(1-outputs_g)    
    
    
    normalized_p = p_x_y_z/torch.sum(p_x_y_z,dim=1,keepdims=True)
#     print(outputs_f[0],outputs_g[0],normalized_p[0])
    return normalized_p

In [7]:
def maximization_step(p_z,focus,classification,data,labels,focus_optimizer,classification_optimizer,Criterion):    
    batch = data.size(0)
    patches = data.size(1)
    focus_optimizer.zero_grad()
    classification_optimizer.zero_grad()
    
    focus_outputs = focus(data)
    data = data.reshape(batch*patches,3,32,32)
    classification_outputs = classification(data) 
    classification_outputs = classification_outputs.reshape(batch,patches,3)
    
    
    #print(focus_outputs,classification_outputs)
    
    loss_focus = calculate_loss_focus(p_z,focus_outputs)
    loss_classification = calculate_loss_classification(p_z,classification_outputs,
                                                        labels,Criterion,patches)
    
    #print("Focus loss",loss_focus.item())
    #print("Classification loss",loss_classification.item())
    loss_focus.backward() 
    loss_classification.backward()
    focus_optimizer.step()
    classification_optimizer.step()
    
    return focus,classification,focus_optimizer,classification_optimizer

In [8]:
def create_mosaic_img(bg_idx,fg_idx,fg,m): 
    """
      bg_idx : list of indexes of background_data[] to be used as background images in mosaic
      fg_idx : index of image to be used as foreground image from foreground data
      fg : at what position/index foreground image has to be stored out of 0-8
    """
    image_list=[]
    j=0
    for i in range(m):  # m value 
        if i != fg:
            image_list.append(background_data[bg_idx[j]])
            j+=1
        else: 
            image_list.append(foreground_data[fg_idx])
            label = foreground_label[fg_idx] - fg1  # minus fg1 because our fore ground classes are fg1,fg2,fg3 but we have to store it as 0,1,2
    #image_list = np.concatenate(image_list ,axis=0)
    image_list = torch.stack(image_list) 
    return image_list,label

In [9]:
fg1, fg2, fg3 = 0,1,2
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=10, shuffle=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=10, shuffle=False)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

foreground_classes = {'plane', 'car', 'bird'}

background_classes = {'cat', 'deer', 'dog', 'frog', 'horse','ship', 'truck'}

# print(type(foreground_classes))

dataiter = iter(trainloader)
background_data=[]
background_label=[]
foreground_data=[]
foreground_label=[]
batch_size=10

for i in tqdm(range(5000)):   #5000*batch_size = 50000 data points
    images, labels = dataiter.next()
    for j in range(batch_size):
        if(classes[labels[j]] in background_classes):
            img = images[j].tolist()
            background_data.append(img)
            background_label.append(labels[j])
        else:
            img = images[j].tolist()
            foreground_data.append(img)
            foreground_label.append(labels[j])
            
foreground_data = torch.tensor(foreground_data)
foreground_label = torch.tensor(foreground_label)
background_data = torch.tensor(background_data)
background_label = torch.tensor(background_label)
print("Foreground Background Data created")



m = 5
desired_num = 40000
mosaic_data =[]      # list of mosaic images, each mosaic image is saved as list of 9 images
fore_idx =[]                   # list of indexes at which foreground image is present in a mosaic image i.e from 0 to 9               
mosaic_label=[]                # label of mosaic image = foreground class present in that mosaic
list_set_labels = [] 
for i in tqdm(range(desired_num)):
    set_idx = set()
    np.random.seed(i)
    bg_idx = np.random.randint(0,35000,m-1)
    set_idx = set(background_label[bg_idx].tolist())
    fg_idx = np.random.randint(0,15000)
    set_idx.add(foreground_label[fg_idx].item())
    fg = np.random.randint(0,m)
    fore_idx.append(fg)
    image_list,label = create_mosaic_img(bg_idx,fg_idx,fg,m)
    mosaic_data.append(image_list)
    mosaic_label.append(label)
    list_set_labels.append(set_idx)
print("Mosaic Data Created")
mosaic_data = torch.stack(mosaic_data)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


100%|██████████| 5000/5000 [00:31<00:00, 161.10it/s]


Foreground Background Data created


100%|██████████| 40000/40000 [00:05<00:00, 7380.93it/s]


Mosaic Data Created


In [10]:
mosaic_data.shape,np.shape(mosaic_label)

(torch.Size([40000, 5, 3, 32, 32]), (40000,))

In [11]:
class MosaicDataset(Dataset):
  """MosaicDataset dataset."""

  def __init__(self, mosaic_list_of_images, mosaic_label, fore_idx):
    """
      Args:
        csv_file (string): Path to the csv file with annotations.
        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied
            on a sample.
    """
    self.mosaic = mosaic_list_of_images
    self.label = mosaic_label
    self.fore_idx = fore_idx

  def __len__(self):
    return len(self.label)

  def __getitem__(self, idx):
    return self.mosaic[idx] , self.label[idx], self.fore_idx[idx]

In [12]:
batch = 250
tr = 30000
msd = MosaicDataset(mosaic_data[0:tr], mosaic_label[0:tr] , fore_idx[0:tr])
train_loader = DataLoader( msd,batch_size= batch ,shuffle=False)

batch = 250
msd1 = MosaicDataset(mosaic_data[tr:], mosaic_label[tr:] , fore_idx[tr:])
test_loader = DataLoader( msd1,batch_size= batch ,shuffle=False)

In [13]:
# torch.manual_seed(12)
# focus = Focus()
# focus = focus.to(device)
# #print(focus.fc1.weight.data)
# # focus.fc1.weight.data = torch.tensor([[0.,0.]])
# torch.manual_seed(12)
# classification = Classification()
# classification = classification.to(device)
# #print(classification.fc1.bias.data)
# # classification.fc1.weight.data = torch.tensor([[0.1,0.1],[-0.1,-0.1],[0.,0.]])
# # classification.fc1.bias.data = torch.tensor([0.,0.,0.])

# Criterion = nn.CrossEntropyLoss(reduction="none") #nn.BCELoss(reduction="none")
# focus_optimizer = optim.SGD(focus.parameters(), lr=0.07,momentum=0.9)
# classification_optimizer = optim.SGD(classification.parameters(),lr=0.07,momentum=0.9)

# # focus_optimizer = optim.Adam(focus.parameters(), lr=0.0001,weight_decay=0.00001)
# # classification_optimizer = optim.Adam(classification.parameters(),lr=0.00005,weight_decay=0.001)

# for i in range(100):
#     focus_epoch_loss = []
#     classification_epoch_loss = []
#     for j,data in enumerate(train_loader):
#         images,labels,foreground_index = data
#         images = images.to(device)
#         labels = labels.to(device)
#         p_z = expectation_step(focus,classification,images,labels)
#         focus,classification,focus_optimizer,classification_optimizer=maximization_step(p_z,focus,classification,
#                                                                                         images,labels,
#                                                                                         focus_optimizer,
#                                                                                         classification_optimizer,
#                                                                                         Criterion)
#         with torch.no_grad():
#             p_z = expectation_step(focus,classification,images,labels)
#             batch = images.size(0)
#             patches = images.size(1)
#             focus_outputs = focus(images)
#             images = images.reshape(batch*patches,3,32,32)
#             classification_outputs = classification(images) # classification returns output after sigmoid/softmax]
#             classification_outputs = classification_outputs.reshape(batch,patches,3)
#             #print(focus_outputs,classification_outputs)
#             loss_focus = calculate_loss_focus(p_z,focus_outputs)
#             loss_classification = calculate_loss_classification(p_z,classification_outputs,
#                                                         labels,Criterion,patches)
#             focus_epoch_loss.append(loss_focus.item())
#             classification_epoch_loss.append(loss_classification.item())
#     print("*"*60)
#     print("Epoch: " + str(i+1)+", Focus Loss: "+str(np.mean(focus_epoch_loss)))
#     print("Epoch: " + str(i+1)+", Classification Loss: "+str(np.mean(classification_epoch_loss))) 
    

In [14]:
torch.manual_seed(12)
focus = Focus()
focus = focus.to(device)


torch.manual_seed(12)
classification = Classification()
classification = classification.to(device)


Criterion = nn.CrossEntropyLoss(reduction="none") #nn.BCELoss(reduction="none")


em_steps = 20


for i in range(em_steps):
    # calculate p_z
    gamma_ = []
    focus_optimizer = optim.SGD(focus.parameters(), lr=0.03,momentum=0.9)
    classification_optimizer = optim.SGD(classification.parameters(),lr=0.03,momentum=0.9)
    for j,data in enumerate(train_loader):
        images,labels,foreground_index = data
        images = images.to(device)
        labels = labels.to(device)
        p_z = expectation_step(focus,classification,images,labels)
        gamma_.append(p_z)
    for epoch in range(10):
        focus_epoch_loss = []
        classification_epoch_loss = []
        for j,data in enumerate(train_loader):
            images,labels,foreground_index = data
            images = images.to(device)
            labels = labels.to(device)
            focus,classification,focus_optimizer,classification_optimizer=maximization_step(gamma_[j],
                                                                                            focus,classification,
                                                                                            images,labels,
                                                                                            focus_optimizer,
                                                                                            classification_optimizer,
                                                                                            Criterion)
            with torch.no_grad():
                batch = images.size(0)
                patches = images.size(1)
                focus_outputs = focus(images)
                images = images.reshape(batch*patches,3,32,32)
                classification_outputs = classification(images) # classification returns output after sigmoid/softmax]
                classification_outputs = classification_outputs.reshape(batch,patches,3)
                #print(focus_outputs,classification_outputs)
                loss_focus = calculate_loss_focus(gamma_[j],focus_outputs)
                loss_classification = calculate_loss_classification(gamma_[j],classification_outputs,
                                                        labels,Criterion,patches)
                focus_epoch_loss.append(loss_focus.item())
                classification_epoch_loss.append(loss_classification.item())
        print("*"*60)
        print("EM Step: "+str(i+1)+" Epoch: " + str(epoch+1)+", Focus Loss: "+str(np.mean(focus_epoch_loss)))
        print("EM Step: "+str(i+1)+" Epoch: " + str(epoch+1)+", Classification Loss: "+str(np.mean(classification_epoch_loss))) 

************************************************************
EM Step: 1 Epoch: 1, Focus Loss: 1.6089806427558264
EM Step: 1 Epoch: 1, Classification Loss: 1.0859565685192745
************************************************************
EM Step: 1 Epoch: 2, Focus Loss: 1.6089772005875906
EM Step: 1 Epoch: 2, Classification Loss: 1.069574592510859
************************************************************
EM Step: 1 Epoch: 3, Focus Loss: 1.6089749147494634
EM Step: 1 Epoch: 3, Classification Loss: 1.0619431545337041
************************************************************
EM Step: 1 Epoch: 4, Focus Loss: 1.608973075946172
EM Step: 1 Epoch: 4, Classification Loss: 1.056315416097641
************************************************************
EM Step: 1 Epoch: 5, Focus Loss: 1.6089714924494425
EM Step: 1 Epoch: 5, Classification Loss: 1.0505760878324508
************************************************************
EM Step: 1 Epoch: 6, Focus Loss: 1.6089700678984324
EM Step: 1 Epoch: 6,

# Evaluation


In [15]:
# method 1
predicted_indexes = []
foreground_index_list = []
prediction_list = []
labels_list = []
with torch.no_grad():
    for j,data in enumerate(train_loader):
        images,labels,foreground_index = data
        images = images.to(device)
        foreground_index_list.append(foreground_index.numpy())
        labels_list.append(labels.numpy())
        batch = images.size(0)
        scores = focus(images)
        indexes = torch.argmax(F.softmax(scores,dim=1),dim=1).cpu().numpy()
        predicted_indexes.append(indexes)
        outputs = F.softmax(classification(images[np.arange(batch),indexes,:]),dim=1)
        prediction = torch.argmax(outputs,dim=1)
        prediction_list.append(prediction.cpu().numpy())
    
predicted_indexes = np.concatenate(predicted_indexes,axis=0)
foreground_index_list = np.concatenate(foreground_index_list,axis=0)
prediction_list = np.concatenate(prediction_list,axis=0)
labels_list = np.concatenate(labels_list,axis=0)
    

print("Focus True",(np.sum(predicted_indexes == foreground_index_list,axis=0).item()/
                    len(foreground_index_list))*100)
accuracy = (np.sum(prediction_list == labels_list,axis=0)/len(labels_list) )*100
print("Accuracy", accuracy)

Focus True 80.23666666666666
Accuracy 94.86


In [16]:
# method 2
predicted_indexes = []
foreground_index_list = []
prediction_list = []
labels_list = []
with torch.no_grad():
    for j,data in enumerate(train_loader):
        images,labels,foreground_index = data
        images = images.to(device)
        batch = images.size(0)
        patches = images.size(1)
        foreground_index_list.append(foreground_index.numpy())
        labels_list.append(labels.numpy())
        batch = images.size(0)
        focus_outputs = F.softmax(focus(images),dim=1)
        indexes = torch.argmax(focus_outputs,dim=1).cpu().numpy()
        predicted_indexes.append(indexes)
        images = images.reshape(batch*patches,3,32,32)
        classification_outputs = F.softmax(classification(images),dim=1)
        classification_outputs = classification_outputs.reshape(batch,patches,3)

        #print(classification_outputs.shape,focus_outputs.shape)

        prediction = torch.argmax(torch.sum(focus_outputs[:,:,None]*classification_outputs,dim=1),dim=1)
        prediction_list.append(prediction.cpu().numpy())
    
predicted_indexes = np.concatenate(predicted_indexes,axis=0)
foreground_index_list = np.concatenate(foreground_index_list,axis=0)
prediction_list = np.concatenate(prediction_list,axis=0)
labels_list = np.concatenate(labels_list,axis=0)
    

print("Focus True",(np.sum(predicted_indexes == foreground_index_list,axis=0).item()/
                    len(foreground_index_list))*100)
accuracy = (np.sum(prediction_list == labels_list,axis=0)/len(labels_list) )*100
print("Accuracy", accuracy)






# method 2
# focus_output = F.softmax(focus(data),dim=1)
# indexes = torch.argmax(F.softmax(focus(data),dim=1),dim=1)[:,0].numpy()
# classification_output = F.softmax(classification(data),dim=2)
# print("Focus True",(np.sum(indexes == fore_idx,axis=0).item()/len(fore_idx))*100)
# prediction = torch.argmax(torch.sum(focus_output*classification_output,dim=1),dim=1)
# accuracy = (torch.sum(prediction == labels,dim=0)/len(labels) )*100
# print("Accuracy", accuracy.item())

Focus True 80.23666666666666
Accuracy 95.61666666666667


In [17]:
print(focus_epoch_loss)

[0.23160898685455322, 0.20825183391571045, 0.26961591839790344, 0.22652606666088104, 0.2822815179824829, 0.26133278012275696, 0.24374254047870636, 0.27502650022506714, 0.24863536655902863, 0.2332628220319748, 0.2713073790073395, 0.2523052990436554, 0.26460257172584534, 0.24730750918388367, 0.25115418434143066, 0.26181459426879883, 0.22098501026630402, 0.2951747179031372, 0.25180527567863464, 0.2349870353937149, 0.27131712436676025, 0.295121431350708, 0.22330091893672943, 0.20936112105846405, 0.25340595841407776, 0.2682759463787079, 0.26291772723197937, 0.22228607535362244, 0.24109819531440735, 0.24506129324436188, 0.2206636369228363, 0.20295432209968567, 0.2812653183937073, 0.24659934639930725, 0.270048588514328, 0.17192624509334564, 0.25391682982444763, 0.26894107460975647, 0.2152675986289978, 0.2658972442150116, 0.23209483921527863, 0.2885556221008301, 0.2475430965423584, 0.2690689265727997, 0.23085753619670868, 0.25203171372413635, 0.2617640793323517, 0.25335970520973206, 0.22597043

> **Test data Evaluation**


In [18]:
# method 1
predicted_indexes = []
foreground_index_list = []
prediction_list = []
labels_list = []
with torch.no_grad():
    for j,data in enumerate(test_loader):
        images,labels,foreground_index = data
        images = images.to(device)
        foreground_index_list.append(foreground_index.numpy())
        labels_list.append(labels.numpy())
        batch = images.size(0)
        scores = focus(images)
        indexes = torch.argmax(F.softmax(scores,dim=1),dim=1).cpu().numpy()
        predicted_indexes.append(indexes)
        outputs = F.softmax(classification(images[np.arange(batch),indexes,:]),dim=1)
        prediction = torch.argmax(outputs,dim=1)
        prediction_list.append(prediction.cpu().numpy())
    
predicted_indexes = np.concatenate(predicted_indexes,axis=0)
foreground_index_list = np.concatenate(foreground_index_list,axis=0)
prediction_list = np.concatenate(prediction_list,axis=0)
labels_list = np.concatenate(labels_list,axis=0)
    

print("Focus True",(np.sum(predicted_indexes == foreground_index_list,axis=0).item()/
                    len(foreground_index_list))*100)
accuracy = (np.sum(prediction_list == labels_list,axis=0)/len(labels_list) )*100
print("Accuracy", accuracy)

Focus True 76.14
Accuracy 84.33


In [19]:
# method 2
predicted_indexes = []
foreground_index_list = []
prediction_list = []
labels_list = []
with torch.no_grad():
    for j,data in enumerate(test_loader):
        images,labels,foreground_index = data
        images = images.to(device)
        batch = images.size(0)
        patches = images.size(1)
        foreground_index_list.append(foreground_index.numpy())
        labels_list.append(labels.numpy())
        batch = images.size(0)
        focus_outputs = F.softmax(focus(images),dim=1)
        indexes = torch.argmax(focus_outputs,dim=1).cpu().numpy()
        predicted_indexes.append(indexes)
        images = images.reshape(batch*patches,3,32,32)
        classification_outputs = F.softmax(classification(images),dim=1)
        classification_outputs = classification_outputs.reshape(batch,patches,3)

        #print(classification_outputs.shape,focus_outputs.shape)

        prediction = torch.argmax(torch.sum(focus_outputs[:,:,None]*classification_outputs,dim=1),dim=1)
        prediction_list.append(prediction.cpu().numpy())
    
predicted_indexes = np.concatenate(predicted_indexes,axis=0)
foreground_index_list = np.concatenate(foreground_index_list,axis=0)
prediction_list = np.concatenate(prediction_list,axis=0)
labels_list = np.concatenate(labels_list,axis=0)
    

print("Focus True",(np.sum(predicted_indexes == foreground_index_list,axis=0).item()/
                    len(foreground_index_list))*100)
accuracy = (np.sum(prediction_list == labels_list,axis=0)/len(labels_list) )*100
print("Accuracy", accuracy)

Focus True 76.14
Accuracy 85.39
